In [1]:
import pandas as pd

# Load the datasets
no_sp_df = pd.read_csv('../log/tmplog_40Mbps_PIAS.csv')
sp_df = pd.read_csv('../log/tmplog_40Mbps_PIAS_SP.csv')

# Display the first few rows of each DataFrame to understand the structure
no_sp_df.head(), sp_df.head()

(                       flow id:  first send:  last arrive:        FCT:  \
 0  11:0:11 11:0:41 10099 100 17     2.003913    235.726320  233.731168   
 1  11:0:11 11:0:41 10097 100 17     2.003863    142.237742  140.252345   
 2  11:0:11 11:0:41 10096 100 17     2.003838    353.086782  351.086255   
 3  11:0:11 11:0:41 10093 100 17     2.003762    150.136419  148.149708   
 4  11:0:11 11:0:41 10092 100 17     2.003737    140.669844  138.684149   
 
    origin_pg:  
 0           3  
 1           3  
 2           3  
 3           3  
 4           3  ,
                        flow id:  first send:  last arrive:        FCT:  \
 0  11:0:11 11:0:41 10099 100 17     2.003913    234.567728  232.570806   
 1  11:0:11 11:0:41 10097 100 17     2.003863    146.710877  144.724041   
 2  11:0:11 11:0:41 10096 100 17     2.003838    350.284524  348.283431   
 3  11:0:11 11:0:41 10093 100 17     2.003762    146.368412  144.381781   
 4  11:0:11 11:0:41 10092 100 17     2.003737    130.411436  128.42643

In [2]:
# Merge the datasets on 'flow id:' column
merged_df = pd.merge(no_sp_df, sp_df, on='flow id:', suffixes=('_no_sp', '_sp'))

# Calculate FCT change and percentage change
merged_df['FCT Change'] = merged_df['FCT:_sp'] - merged_df['FCT:_no_sp']
merged_df['FCT % Change'] = (merged_df['FCT Change'] / merged_df['FCT:_no_sp']) * 100

# Display the new dataframe with changes
merged_df[['flow id:', 'FCT:_no_sp', 'FCT:_sp', 'FCT Change', 'FCT % Change']].head()


,flow id:,FCT:_no_sp,FCT:_sp,FCT Change,FCT % Change
0,11:0:11 11:0:41 10099 100 17,233.731168,232.570806,-1.160362,-0.496452
1,11:0:11 11:0:41 10097 100 17,140.252345,144.724041,4.471697,3.188322
2,11:0:11 11:0:41 10096 100 17,351.086255,348.283431,-2.802825,-0.798329
3,11:0:11 11:0:41 10093 100 17,148.149708,144.381781,-3.767927,-2.543324
4,11:0:11 11:0:41 10092 100 17,138.684149,128.426431,-10.257718,-7.396460


In [3]:
merged_df

,flow id:,first send:_no_sp,last arrive:_no_sp,FCT:_no_sp,origin_pg:_no_sp,first send:_sp,last arrive:_sp,FCT:_sp,origin_pg:_sp,FCT Change,FCT % Change
0,11:0:11 11:0:41 10099 100 17,2.003913,235.726320,233.731168,3,2.003913,234.567728,232.570806,3,-1.160362,-0.496452
1,11:0:11 11:0:41 10097 100 17,2.003863,142.237742,140.252345,3,2.003863,146.710877,144.724041,3,4.471697,3.188322
2,11:0:11 11:0:41 10096 100 17,2.003838,353.086782,351.086255,3,2.003838,350.284524,348.283431,3,-2.802825,-0.798329
3,11:0:11 11:0:41 10093 100 17,2.003762,150.136419,148.149708,3,2.003762,146.368412,144.381781,3,-3.767927,-2.543324
4,11:0:11 11:0:41 10092 100 17,2.003737,140.669844,138.684149,3,2.003737,130.411436,128.426431,3,-10.257718,-7.396460
...,...,...,...,...,...,...,...,...,...,...,...
295,11:0:31 11:0:41 10053 100 17,2.001550,140.754664,138.771869,2,2.001550,135.208116,133.206841,2,-5.565028,-4.010199
296,11:0:31 11:0:41 10054 100 17,2.001594,218.547536,216.555273,2,2.001594,170.054559,168.068011,2,-48.487262,-22.390248
297,11:0:31 11:0:41 10008 100 17,2.000225,127.367751,125.384738,1,2.000225,127.378376,125.378340,1,-0.006398,-0.005103
298,11:0:31 11:0:41 10058 100 17,2.002083,113.785514,111.803086,3,2.002083,113.795929,111.811019,3,0.007933,0.007096


In [4]:
# Group by origin priority group and calculate the required statistics
group_stats = merged_df.groupby('origin_pg:_no_sp').agg(
    FCT_Total_No_SP=pd.NamedAgg(column='FCT:_no_sp', aggfunc='sum'),
    FCT_Total_SP=pd.NamedAgg(column='FCT:_sp', aggfunc='sum'),
    Total_FCT_Change=pd.NamedAgg(column='FCT Change', aggfunc='sum'),
    Mean_FCT_Change=pd.NamedAgg(column='FCT Change', aggfunc='mean'),
    Count_FCT_Decrease=pd.NamedAgg(column='FCT Change', aggfunc=lambda x: (x < 0).sum()),
    Total_Flows=pd.NamedAgg(column='FCT Change', aggfunc='size')
)

# Calculate percentage of flows with decreased FCT
group_stats['Percentage_FCT_Decrease'] = (group_stats['Count_FCT_Decrease'] / group_stats['Total_Flows']) * 100

# Display the summarized statistics
group_stats.reset_index()


,origin_pg:_no_sp,FCT_Total_No_SP,FCT_Total_SP,Total_FCT_Change,Mean_FCT_Change,Count_FCT_Decrease,Total_Flows,Percentage_FCT_Decrease
0,1,15490.317880,15494.905581,4.587700,0.033983,105,135,77.777778
1,2,20536.069648,19805.142160,-730.927488,-24.364250,23,30,76.666667
2,3,61268.012142,60989.455859,-278.556283,-2.063380,59,135,43.703704
